# Reading and cleaning the data

We start by reading the data from disk.

In [14]:
import pandas as pd 
import numpy as np
%precision %.2f
pd.options.display.float_format = '{:,.2f}'.format


# read raw csv file
ordersFile = "D:\\Universitaet\\Lehrstuhl Koeln Grahl\\02 Forschung\\Aktiv\\Pegasus\\data\\orders.txt"
data = pd.read_csv(ordersFile, sep="\t", parse_dates=['dateOfOrder']) # read file
data[:5] # verify by showing top 5 lines

,id,dateOfOrder,customerId,typeOfOrder,id.1,orderId,articleId,title,amount,price,discount
0,190,2004-05-11 16:47:01,29957,ORDER,743.00,190.00,HER00500,Fantasy Hero,1.00,38.40,nan
1,191,2004-05-11 16:51:35,30247,ORDER,744.00,191.00,FFG0WC01,Warcraft DEUTSCH,1.00,39.95,nan
2,191,2004-05-11 16:51:35,30247,ORDER,745.00,191.00,EAG00004,Civilization DEUTSCH,1.00,49.99,nan
3,192,2004-05-11 17:29:35,29895,ORDER,746.00,192.00,GSG0F132,Unter Druck,1.00,10.10,nan
4,192,2004-05-11 17:29:35,29895,ORDER,747.00,192.00,GIV02030,Konzernreport #2: Lazarus,1.00,9.50,nan


In [15]:
data.describe()

,id,customerId,id.1,orderId,amount,price,discount
count,"1,454,437.00","1,454,437.00","1,454,433.00","1,454,433.00","1,454,433.00","1,394,006.00",23.00
mean,"149,861.58","14,642.62","728,021.01","149,861.90",2.41,12.46,24.78
std,"89,050.63","7,992.95","419,897.83","89,050.53",89.82,14.22,1.04
min,190.00,-1.00,743.00,190.00,"-36,138.00",-1.00,20.00
25%,"70,150.00","11,239.00","364,379.00","70,151.00",1.00,4.77,25.00
50%,"149,593.00","12,061.00","728,033.00","149,594.00",1.00,8.24,25.00
75%,"229,428.00","13,477.00","1,091,641.00","229,428.00",2.00,15.99,25.00
max,"296,838.00","108,272.00","1,455,320.00","296,838.00","90,910.00","1,532.86",25.00


Strange things: 
1. Discounts only available for 23 observations
1. Negative prices
1. These vars have fewer values than total number of obs: `orderId`, `amount`, `price`
1. Negative customer Id

What I do: 
1. I drop `discount` var. 
1. I remove all obs with `prices<0`.
1. I drop all obs that have missing values.
1. I keep only customers with a `customerId > 0`

In [16]:
data = data.drop(['discount'], axis = 1) # drop discount column
data.dropna(inplace = True) # remove rows that have a missing value
data = data.loc[data['price'] >= 0] # remove orders with negative prices
data = data.loc[data['customerId'] > 0] # remove negative customer IDs
data.describe() # once more

,id,customerId,id.1,orderId,amount,price
count,"1,384,879.00","1,384,879.00","1,384,879.00","1,384,879.00","1,384,879.00","1,384,879.00"
mean,"149,394.76","14,685.68","725,758.67","149,394.76",2.51,12.42
std,"88,569.98","7,780.82","417,416.32","88,569.98",10.24,14.16
min,190.00,"10,000.00",743.00,190.00,1.00,0.00
25%,"70,261.00","11,256.00","364,897.50","70,261.00",1.00,4.75
50%,"148,753.00","12,061.00","725,065.00","148,753.00",1.00,8.22
75%,"227,849.00","13,469.00","1,083,495.50","227,849.00",2.00,15.93
max,"296,838.00","108,272.00","1,455,320.00","296,838.00","3,000.00","1,532.86"


Count is identical for all vars. Prices >= 0. Discounts removed.

In [17]:
pd.unique(data['typeOfOrder']) # there are several possible entries for the order column
data = data.loc[data['typeOfOrder'] == "ORDER"] # keep rows that are ORDERS
data.describe()

,id,customerId,id.1,orderId,amount,price
count,"1,384,704.00","1,384,704.00","1,384,704.00","1,384,704.00","1,384,704.00","1,384,704.00"
mean,"149,399.87","14,683.32","725,781.68","149,399.87",2.51,12.42
std,"88,573.15","7,778.30","417,432.45","88,573.15",10.24,14.16
min,190.00,"10,000.00",743.00,190.00,1.00,0.00
25%,"70,256.00","11,256.00","364,853.75","70,256.00",1.00,4.75
50%,"148,763.00","12,061.00","725,120.50","148,763.00",1.00,8.22
75%,"227,858.00","13,469.00","1,083,541.25","227,858.00",2.00,15.93
max,"296,838.00","108,272.00","1,455,320.00","296,838.00","3,000.00","1,532.86"


# Customer descriptives



In [20]:
customerIds = pd.unique(data['customerId']) # get all customerIds
numberOfCustomers = len(customerIds) # count
numberOfOrders = data.shape[0] # get number of rows of order dataset
productIds = pd.unique(data['articleId']) # get all productIds
numberOfProducts = len(productIds) # count

print ("The number of customers is", numberOfCustomers)
print ("The number of orders is", numberOfOrders)
print ("The number of products is", numberOfProducts)

The number of customers is 17469
The number of orders is 1384704
The number of products is 38116


I now aggregate data on customer level in order to describe customer behavior.  
I compute order value as `amount x price`.

In [21]:
data['orderValue'] = data['price'] * data['amount'] # compute order value 
# group by customer and compute some statistics
customerOrders = data.groupby('customerId')['orderValue'].agg(['sum', 'count'])  
d = customerOrders.describe()
print("On average, customers have placed", d['count']['mean'], "orders.")
print("On average, customers have purchased for", d['sum']['mean'], "Euro")

On average, customers have placed 79.2663575477 orders.
On average, customers have purchased for 2514.20501966 Euro
